In [ ]:
import json
from jupyter_client import kernelspec
from papermill import execute_notebook
from sys import executable
from ._collections import singleton


def current_kernel():
    return singleton([ 
        name
        for name, kernel_dir
        in kernelspec.find_kernel_specs().items()
        if json.load(open(f'{kernel_dir}/kernel.json','r'))['argv'][0] == executable
    ])


def execute(input, output=None, nest_asyncio=True, *args, **kwargs):
    output = output or input
    if 'kernel' in kwargs:
        kernel_name = kwargs.pop('kernel')
    else:
        kernel_name = current_kernel()
    execute_notebook(
        str(input),
        str(output),
        *args,
        nest_asyncio=nest_asyncio,
        kernel_name=kernel_name,
        parameters=kwargs,
    )